# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [637]:
problem = '''The PRODA, S.A. industrial products firm has to face the problem of scheduling
the weekly production of its three products (P1, P2 and P3). These products are
sold to large industrial firms and PRODA, S.A. wishes to supply its products in
quantities that are more profitable for it.

Each product entails three operations contributing to the costs: smelting; mechanisation; assembly and
packaging. The smelting operations for products P1 and P2 could be subcontracted, but the smelting operation for product P3 requires special equipment, thus
preventing the use of subcontracts. PRODA also want to know, how much they should subcontract.

For product P1 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.30$
- subcontracted smelting: 0.50$
- mechanisation: 0.20$
- Assembly and packaging: 0.3$
The unit sales price is 1.50$.

For product P2 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.50$
- subcontracted smelting: 0.60$
- mechanisation: 0.10$
- Assembly and packaging: 0.20$
The unit sales price is 1.80$.

For product P3 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.40$
- mechanisation: 0.27$
- Assembly and packaging: 0.20$
The unit sales price is 1.97$.

Each unit of product P1 requires 6 min of smelting time (if performed at PRODA, S.A.), 6 min of mechanisation time and 3 min of assembly and packaging time, respectively. For product P2, the times are 10, 3 and 2 min, respectively. One unit of product P3 needs 8 min of smelting time, 8 min of mechanisation and 2 min for assembly and packaging. PRODA, S.A. has weekly capacities of 8,000 min of smelting time, 12,000 min of mechanisation time and 10,000 min of assembly and packaging time.
The objective is to maximize weekly profits.'''

## 2. Ask for parameters

In [727]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [728]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

**Production Quantities:**

*  $x_1$: Number of units of product P1 produced and sold weekly.
*  $x_2$: Number of units of product P2 produced and sold weekly.
*  $x_3$: Number of units of product P3 produced and sold weekly.

**Subcontracting Quantities:**

*  $s_1$: Number of units of product P1 subcontracted for smelting weekly.
*  $s_2$: Number of units of product P2 subcontracted for smelting weekly. 


# 2. Ask for objective

In [743]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [744]:
Markdown(response2.text)

$$\max  \  1.2x_1 - 0.2s_1 + 1.3x_2 - 0.1s_2 + 1.57x_3$$ 


# 3. Ask for constraints

In [745]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [746]:
Markdown(response3.text)

**Constraints:**

* **Production Capacity:**
    * Smelting: $6(x_1 - s_1) + 10(x_2 - s_2) + 8x_3 \le 8000$ 
    * Mechanization: $6x_1 + 3x_2 + 8x_3 \le 12000$
    * Assembly and Packaging: $3x_1 + 2x_2 + 2x_3 \le 10000$

* **Subcontracting Limits:**
    *  $s_1 \le x_1$ (Can't subcontract more than what's produced)
    *  $s_2 \le x_2$ (Can't subcontract more than what's produced)

* **Non-Negativity:**
    *  $x_1, x_2, x_3, s_1, s_2 \ge 0$ (Can't produce or subcontract negative quantities) 


## 3. Generate the pyomo code

In [747]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [748]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Define variables
model.x1 = Var(domain=NonNegativeReals)  # Product P1 produced and sold
model.x2 = Var(domain=NonNegativeReals)  # Product P2 produced and sold
model.x3 = Var(domain=NonNegativeReals)  # Product P3 produced and sold
model.s1 = Var(domain=NonNegativeReals)  # Product P1 subcontracted
model.s2 = Var(domain=NonNegativeReals)  # Product P2 subcontracted

# Define the objective function
model.profit = Objective(
    expr=1.2 * model.x1 - 0.2 * model.s1 + 1.3 * model.x2 - 0.1 * model.s2 + 1.57 * model.x3,
    sense=maximize
)

# Define constraints
model.smelting = Constraint(
    expr=6 * (model.x1 - model.s1) + 10 * (model.x2 - model.s2) + 8 * model.x3 <= 8000
)
model.mechanization = Constraint(
    expr=6 * model.x1 + 3 * model.x2 + 8 * model.x3 <= 12000
)
model.assembly_packaging = Constraint(
    expr=3 * model.x1 + 2 * model.x2 + 2 * model.x3 <= 10000
)
model.subcontracting_p1 = Constraint(expr=model.s1 <= model.x1)
model.subcontracting_p2 = Constraint(expr=model.s2 <= model.x2)

# Solve the model
SolverFactory('glpk').solve(model)  # You can choose a different solver if needed

# Print the results
print("Optimal Solution:")
print(f"  x1 (P1 Production): {model.x1.value:.2f}")
print(f"  x2 (P2 Production): {model.x2.value:.2f}")
print(f"  x3 (P3 Production): {model.x3.value:.2f}")
print(f"  s1 (P1 Subcontracted): {model.s1.value:.2f}")
print(f"  s2 (P2 Subcontracted): {model.s2.value:.2f}")
print(f"  Total Profit: ${model.profit.expr():.2f}") 
``` 


## 4. Input problem data and try running the generated code

In [749]:
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Define variables
model.x1 = Var(domain=NonNegativeReals)  # Product P1 produced and sold
model.x2 = Var(domain=NonNegativeReals)  # Product P2 produced and sold
model.x3 = Var(domain=NonNegativeReals)  # Product P3 produced and sold
model.s1 = Var(domain=NonNegativeReals)  # Product P1 subcontracted
model.s2 = Var(domain=NonNegativeReals)  # Product P2 subcontracted

# Define the objective function
model.profit = Objective(
    expr=1.2 * model.x1 - 0.2 * model.s1 + 1.3 * model.x2 - 0.1 * model.s2 + 1.57 * model.x3,
    sense=maximize
)

# Define constraints
model.smelting = Constraint(
    expr=6 * (model.x1 - model.s1) + 10 * (model.x2 - model.s2) + 8 * model.x3 <= 8000
)
model.mechanization = Constraint(
    expr=6 * model.x1 + 3 * model.x2 + 8 * model.x3 <= 12000
)
model.assembly_packaging = Constraint(
    expr=3 * model.x1 + 2 * model.x2 + 2 * model.x3 <= 10000
)
model.subcontracting_p1 = Constraint(expr=model.s1 <= model.x1)
model.subcontracting_p2 = Constraint(expr=model.s2 <= model.x2)

# Solve the model
SolverFactory('glpk').solve(model)  # You can choose a different solver if needed

# Print the results
print("Optimal Solution:")
print(f"  x1 (P1 Production): {model.x1.value:.2f}")
print(f"  x2 (P2 Production): {model.x2.value:.2f}")
print(f"  x3 (P3 Production): {model.x3.value:.2f}")
print(f"  s1 (P1 Subcontracted): {model.s1.value:.2f}")
print(f"  s2 (P2 Subcontracted): {model.s2.value:.2f}")
print(f"  Total Profit: ${model.profit.expr():.2f}") 

Optimal Solution:
  x1 (P1 Production): 0.00
  x2 (P2 Production): 4000.00
  x3 (P3 Production): 0.00
  s1 (P1 Subcontracted): 0.00
  s2 (P2 Subcontracted): 3200.00
  Total Profit: $4880.00


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [750]:
print(response.text)

## Variables:

**Production Quantities:**

*  $x_1$: Number of units of product P1 produced and sold weekly.
*  $x_2$: Number of units of product P2 produced and sold weekly.
*  $x_3$: Number of units of product P3 produced and sold weekly.

**Subcontracting Quantities:**

*  $s_1$: Number of units of product P1 subcontracted for smelting weekly.
*  $s_2$: Number of units of product P2 subcontracted for smelting weekly. 



In [751]:
print(response2.text)

$$\max  \  1.2x_1 - 0.2s_1 + 1.3x_2 - 0.1s_2 + 1.57x_3$$ 



In [752]:
print(response3.text)

**Constraints:**

* **Production Capacity:**
    * Smelting: $6(x_1 - s_1) + 10(x_2 - s_2) + 8x_3 \le 8000$ 
    * Mechanization: $6x_1 + 3x_2 + 8x_3 \le 12000$
    * Assembly and Packaging: $3x_1 + 2x_2 + 2x_3 \le 10000$

* **Subcontracting Limits:**
    *  $s_1 \le x_1$ (Can't subcontract more than what's produced)
    *  $s_2 \le x_2$ (Can't subcontract more than what's produced)

* **Non-Negativity:**
    *  $x_1, x_2, x_3, s_1, s_2 \ge 0$ (Can't produce or subcontract negative quantities) 



In [753]:
print(response4.text)

```python
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Define variables
model.x1 = Var(domain=NonNegativeReals)  # Product P1 produced and sold
model.x2 = Var(domain=NonNegativeReals)  # Product P2 produced and sold
model.x3 = Var(domain=NonNegativeReals)  # Product P3 produced and sold
model.s1 = Var(domain=NonNegativeReals)  # Product P1 subcontracted
model.s2 = Var(domain=NonNegativeReals)  # Product P2 subcontracted

# Define the objective function
model.profit = Objective(
    expr=1.2 * model.x1 - 0.2 * model.s1 + 1.3 * model.x2 - 0.1 * model.s2 + 1.57 * model.x3,
    sense=maximize
)

# Define constraints
model.smelting = Constraint(
    expr=6 * (model.x1 - model.s1) + 10 * (model.x2 - model.s2) + 8 * model.x3 <= 8000
)
model.mechanization = Constraint(
    expr=6 * model.x1 + 3 * model.x2 + 8 * model.x3 <= 12000
)
model.assembly_packaging = Constraint(
    expr=3 * model.x1 + 2 * model.x2 + 2 * model.x3 <= 10000
)
model.subcontracting_p1 = 